Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [2]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [3]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [4]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        """
        Args:
            num_rows (int): number of piles
            k (int, optional): maximum number of objects nimmable each time. Defaults to None (any amount).
        """
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k, f"{num_objects=}, {self._k=}"
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [5]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row]) if state._k is None else min(random.randint(1, state.rows[row]), state._k)
    return Nimply(row, num_objects)


In [6]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the smallest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c+1 if state._k is None else min(c + 1, state._k))]
    return max(possible_moves, key=lambda m: (-m[0], m[1]))

In [7]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c+1 if raw._k is None else min(c + 1, raw._k))):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    logging.debug(pformat(f"{analysis['possible_moves']}"))
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [8]:
logging.getLogger().setLevel(logging.INFO)

strategy = (gabriele, optimal)

nim = Nim(5,5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.info(f"ply: player {player} ({strategy[player].__qualname__}) \t plays {ply} -> {nim} ({nim_sum(nim)})")
    player = 1 - player
logging.info(f"status: Player {player} ({strategy[player].__qualname__}) won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 (gabriele) 	 plays (0, 1) -> <0 3 5 7 9> (8)
INFO:root:ply: player 1 (optimal) 	 plays Nimply(row=4, num_objects=3) -> <0 3 5 7 6> (7)
INFO:root:ply: player 0 (gabriele) 	 plays (1, 3) -> <0 0 5 7 6> (4)
INFO:root:ply: player 1 (optimal) 	 plays Nimply(row=4, num_objects=1) -> <0 0 5 7 5> (7)
INFO:root:ply: player 0 (gabriele) 	 plays (2, 4) -> <0 0 1 7 5> (3)
INFO:root:ply: player 1 (optimal) 	 plays Nimply(row=3, num_objects=2) -> <0 0 1 5 5> (1)
INFO:root:ply: player 0 (gabriele) 	 plays (2, 1) -> <0 0 0 5 5> (0)
INFO:root:ply: player 1 (optimal) 	 plays Nimply(row=4, num_objects=2) -> <0 0 0 5 3> (6)
INFO:root:ply: player 0 (gabriele) 	 plays (3, 4) -> <0 0 0 1 3> (2)
INFO:root:ply: player 1 (optimal) 	 plays Nimply(row=4, num_objects=3) -> <0 0 0 1 0> (1)
INFO:root:ply: player 0 (gabriele) 	 plays (3, 1) -> <0 0 0 0 0> (0)
INFO:root:status: Player 1 (optimal) won!


# Evolutionary-Based Approach

In [45]:
from dataclasses import dataclass, asdict, field
from typing import Literal, TypedDict, Any
import random

Allele: TypedDict = {
    "prefer_rows": {"type": "discrete", "value": [0, 1, -1]},
    "percent_to_take": {"type": "continous", "value": [0, 1]},
}

def random_allele_value(key):
    v: dict[str, Any] = Allele[key]
    if v.get("type") == "discrete":
        return random.choice(v["value"])
    elif v.get("type") == "continous":
        start, stop = v.get("value")
        size = stop-start
        return (random.random() * size) + start


@dataclass(frozen=True)
class Genome:
    prefer_rows: Literal[0, 1, -1] = field(default_factory=lambda: random_allele_value("prefer_rows"))
    """-1 favours smaller rows, 1 bigger, 0 indifferent"""
    percent_to_take: float = field(default_factory=lambda: random_allele_value("percent_to_take"))
    """Range: [0, 1]"""

    def mutate(g1: "Genome") -> "Genome":
        """Alters one single gene of the starting genome

        Args:
            g1 (Genome): Starting genome (never altered)

        Returns:
            Genome: Mutated genome
        """
        d = asdict(g1)
        rand_attr_to_change: str = random.choice(list(d.keys()))
        d[rand_attr_to_change] = random_allele_value(rand_attr_to_change)
        return Genome(**d)

    def crossover(g1: "Genome", g2: "Genome") -> "Genome":
        d1, d2 = asdict(g1), asdict(g2)
        child = dict()
        for field in d1.keys():
            child[field] = d1[field] if random.random() < 0.5 else d2[field]

        return Genome(**child)
    
d = Genome()
e = Genome()
print(d, e, d.crossover(e).mutate())

Genome(prefer_rows=-1, percent_to_take=0.5372056503465726) Genome(prefer_rows=0, percent_to_take=0.4008275725094904) Genome(prefer_rows=-1, percent_to_take=0.5372056503465726)


In [ ]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}
